# Title: "How Does a Bike-Share Navigate Speedy Success?"
Author: William

***

## **Scenario**

<center><img src = "scripts/logo.png" alt = "logo" width = "50%"/></center>

The Cyclistic is a bike-sharing company. They desire better to understand the
customer behavior to create new marketing strategy.

In order to do that, the company provide their dataset to a data analyst team
analyze and come up with data insights. Then, the executive team will inspect
the results and further make data-driven decision-making.

The Cyclistic has more thousands of bikes divided in different categories such
as classic, reclining, electric. There are two types of customers, the ones with
annual membership and the one with a pass (single-ride or full-day).

***

## **Ask**
In this phase, we define the problem and the stakeholders.

### Business task
The business task is to determine how different the customers use the company's
bikes.

### Stakeholders
The stakeholders are:

  * Primary: director of marketing
  * Secondary: marketing analytics team

***

## **Prepare**
Now, the focus is to collect, store, and evaluate the datasets

### Information about the dataset
Regarding the dataset, we have the following information:

  * It is external data stored in the [cloud](https://divvy-tripdata.s3.amazonaws.com/index.html)
  * It used the long data format
  * First-party data (reliable and original)
  * There is a file for each past 12 months (current)
  * It is distributed under a [license](https://www.divvybikes.com/data-license-agreement)
  * There is not Personally Identifiable Information (PII)
  
### Data acquisition process
The following process is utilized:

  * Each dataset is downloaded
  * Appropriately stored in a folder for original datasets
  
### Identifying issues
In order to identify issues with the data, we:

  * Evaluate the ride length and spot unusual observations
  * Filtered the data and identified missing values
  * Sorted the data and found inconsistent attribute format

***

## **Process**
Here, we perform the data cleaning, ensure integrity, and that it is complete,
correct, and relevant.

### Tools
We selected tools to perform specific tools, as described next.

  * Spreadsheets: perform initial data inspection
  * R: develop scripts to clean, transform, organize, and summarize the datasets
  * Tableau: create data visualizations
  * R Notebook: provide a complete report of the data analysis process

### Libraries utilized
Here is a list of the R packages utilized:

  * Tidyverse
  * Lubridate
  * ReadR
  * Tibble
  * DplyR
  
They can be installed and loaded as follows.

In [ ]:
# Installing libraries
#install.packages("tidyverse")
#install.packages("lubridate")
#install.packages("readr")
#install.packages("tibble")
#install.packages("dplyr")

# Loading libraries
library("tidyverse")
library("lubridate")
library("readr")
library("tibble")
library("dplyr")

### Cleaning all the CSV files and saving in the cleaned datasets folder
In this step, we perform the following tasks:

  * Discard ride id, latitude, and longitude columns
  * Remove rows with missing values
  * Remove duplicated rows
  * Add ride length column (in seconds)
  * Add day of week column (1 means Sunday and 7 means Saturday)
  * Remove rides with less than 5 minutes
  * Remove rides with more than 24 hours
  * Fix rows with negative ride length
  * Rename the member casual column to membership
  * Save the cleaned datasets

In [ ]:
# Cleaning data

# List all the CSV files
file_names <- list.files(path = "../datasets/original_datasets/",
                         pattern = "*.csv", full.names = FALSE,
                         recursive = FALSE)

# Clean all CSV files
for(name in file_names){
  # Read CSV file
  data <- read_csv(paste("../datasets/original_datasets/", name,
                         sep = ""))[, c(2:8,13)]

  # Drop rows with missing values
  data <- drop_na(data)
  
  # Drop duplicates
  data <- distinct(data)
  
  # Calculate the ride length
  ride_length <- difftime(data$ended_at, data$started_at, units = "secs")
  
  # Calculate the day of week (1 = Sunday, 7 = Saturday)
  day_of_week <- wday(data$started_at, week_start = 7)
  
  # Add ride_length and day_of_week columns to the data
  data <- add_column(data, ride_length, day_of_week, .after = "ended_at")
  
  # Filter the ride_length to be at least 5 min and at most 24 hours
  data <- filter(data, 300 <= abs(data$ride_length) &
                   abs(data$ride_length) <= 86400)
  
  # Find the index of ride_length, started_at, and ended_at columns
  index_ride <- grep("ride_length", colnames(data))
  index_started <- grep("started_at", colnames(data))
  index_ended <- grep("ended_at", colnames(data))
  
  # If the ride_length is negative, change the started_at and ended_at
  # columns and make ride_length positive
  for(row_number in 1:nrow(data)){
    if(data[row_number, index_ride] < 0){
      tmp <- data[row_number, index_started]
      data[row_number, index_started] <- data[row_number, index_ended]
      data[row_number, index_ended] <- tmp
      data[row_number, index_ride] = -1 * data[row_number, index_ride]
    }
  }
  
  # Rename member_casual column to membership
  data <- rename(data, membership = member_casual)
  
  # Saves data as a CSV file
  write_csv(data, paste("../datasets/cleaned_datasets/", name, sep = ""))
}

***

## **Analyze**

Now, we analyze the data to find patterns, relationship, or trends. Then, we
write down all the insights.

### Aggregate data

We aggregate all the 12 datasets into a single file. We assume the datasets do
not contain duplicates with each other.

In [ ]:
# Aggregating data

# List all the CSV files
file_names <- list.files(path = "../datasets/cleaned_datasets/",
                         pattern = "*.csv", full.names = FALSE,
                         recursive = FALSE)

# Save the first file and header
aggregated_data <- read_csv(paste("../datasets/cleaned_datasets/",
                                  file_names[1], sep = ""))
write_csv(aggregated_data, "../datasets/analyzed_datasets/aggregated_data.csv")

# Aggregate all CSV files
for(name in file_names[2:length(file_names)]){
  # Read CSV file
  data <- read_csv(paste("../datasets/cleaned_datasets/", name, sep = ""))
  
  # Aggregating datasets
  #aggregated_data <- union_all(aggregated_data, data)
  
  # Drop duplicates
  #aggregated_data <- distinct(aggregated_data)
  
  # Save aggregated data as a CSV file
  write_csv(data, "../datasets/analyzed_datasets/aggregated_data.csv",
            append = TRUE)
}

### Summarize data
In this step, we create several approaches to the dataset, as follows.

  * Identify the top 10 stations that members start a ride
    + Include the values for casuals on the same stations
  * Identify the top 10 stations that casuals start a ride
    + Include the values for members on the same stations
  * Summarize for member as casuals:
    + Minimum ride length
    + Mean ride length
    + Maximum ride length
    + Mode of the day of week
    + Frequency of rides
  * Frequency of rides by day
  * Frequency of rideables used
  * Frequency of rides by month
  * Frequency of rides by period of the day
  * Organize the columns of each item above
  * Save each approach in a separate file

In [ ]:
# Create mode function
mode_func <- function(data){
  unique_data <- unique(data)
  tabulate_data <- tabulate(match(data, unique_data))
  unique_data[tabulate_data == max(tabulate_data)]
}

# Create save function
save_file <- function(data, file_name){
  write_csv(data, paste("../datasets/analyzed_datasets/", file_name, ".csv",
                        sep = ""))
}

# Summarizing data

# Read the aggregated data
data <- read_csv("../datasets/analyzed_datasets/aggregated_data.csv",
                 col_types = cols(start_station_id = col_character(),
                                  end_station_id = col_character()))

# Identify the top 10 stations that members start a ride
member <- filter(data, membership == "member")
member <- count(member, start_station_id, name = "frequency", sort = TRUE)
member$membership = "member"
top_member <- head(member, 10)

# Identify the top 10 stations that casuals start a ride
casual <- filter(data, membership == "casual")
casual <- count(casual, start_station_id, name = "frequency", sort = TRUE)
casual$membership = "casual"
top_casual <- head(casual, 10)

# Complement the top stations of members with the casual
top_member_complete <- inner_join(select(top_member, "start_station_id"),
                                  unique(casual), by = "start_station_id")
top_member_complete <- union(top_member, top_member_complete)

# Complement the top stations of casual with the members
top_casual_complete <- inner_join(select(top_casual, "start_station_id"),
                                  unique(member), by = "start_station_id")
top_casual_complete <- union(top_casual, top_casual_complete)

# List the stations names
stations_names <- unique(select(data, "start_station_id", "start_station_name"))

# Add the stations names to top stations of members
top_member_complete <- inner_join(top_member_complete, stations_names,
                                  by = "start_station_id")
top_member_complete <- top_member_complete[, c(1, 4, 2, 3)]

# Add the stations names to top stations of casuals
top_casual_complete <- inner_join(top_casual_complete, stations_names,
                                  by = "start_station_id")
top_casual_complete <- top_casual_complete[, c(1, 4, 2, 3)]

# Save the frequency of top stations of members data as a CSV file
save_file(top_member_complete, "top_member")

# Save the frequency of top stations of casuals data as a CSV file
save_file(top_casual_complete, "top_casual")

# Group data by member relationship
data <- group_by(data, membership)

# Summarize the data
summarized_data <- summarize(data, min_ride_length = min(ride_length),
                             mean_ride_length = mean(ride_length),
                             max_ride_length = max(ride_length),
                             mode_day_week = mode_func(day_of_week))
frequency_rides <- count(data, membership, name = "frequency_rides")
summarized_data <- inner_join(summarized_data, frequency_rides,
                              by = "membership")
summarized_data <- summarized_data[, c(2, 3, 4, 5, 6, 1)]
save_file(summarized_data, "summarized_data")

# Frequency of rides by day
frequency_week <- count(data, day_of_week, name = "frequency")
frequency_week$day_of_week <- recode(frequency_week$day_of_week, "1" = "Sunday",
                                     "2" = "Monday", "3" = "Tuesday",
                                     "4" = "Wednesday", "5" = "Thursday",
                                     "6" = "Friday", "7" = "Saturday")
frequency_week <- rename(frequency_week, weekday = day_of_week)
frequency_week <- frequency_week[, c(2, 3, 1)]

# Save the frequency of week data as a CSV file
save_file(frequency_week, "frequency_week")

# Frequency of rideables
frequency_rideables <- count(data, rideable_type, name = "frequency")
frequency_rideables$rideable_type <- str_replace(frequency_rideables$rideable_type, "_", " ")
frequency_rideables <- frequency_rideables[, c(2, 3, 1)]

# Save the frequency of rideables data as a CSV file
save_file(frequency_rideables, "frequency_rideables")

# Splitting datetime to date and time
tmp <- as_date(data$started_at)
#data$year <- format(tmp, "%Y")
#data$month <- format(tmp, "%m")
data$month_year <- format(tmp, "%m/%Y")
#data$day <- format(tmp, "%d")
#data$time <- format(data$started_at, format = "%H:%M:%S")
data$hour <- as.numeric(format(data$started_at, format = "%H"))

# Frequency of rides by month
frequency_rides_month <- count(data, month_year, name = "frequency")
frequency_rides_month <- frequency_rides_month[, c(2, 3, 1)]

# Save the frequency of rides by month data as a CSV file
save_file(frequency_rides_month, "frequency_rides_months")

# Frequency of rides by period of the day
period_time <- hour(hm("00:00", "06:00", "12:00", "18:00", "23:59"))
period_name <- c("Night", "Morning", "Afternoon", "Evening")
data$period <- cut(x = data$hour, breaks = period_time,
                       labels = period_name, include.lowest = TRUE)
frequency_period <- count(data, period, name = "frequency")
frequency_period <- frequency_period[, c(2, 3, 1)]

# Save the frequency of period of day data as a CSV file
save_file(frequency_period, "frequency_rides_period")

### Analysis summary

Analyzing the summarized data, we have the following findings:

  * Most stations that members and casuals start their rides, but there are in
  common
  * The minimum and maximum ride length are similar between the different
  customers, but the casuals have longer rides (~2.2x) in average
  * The number of members riders is almost 1.3x the casuals ones.
  * Both types of customers frequently ride in Saturday
  * The casual riders often rent bikes on the weekend
  * The number of member rides increases from Sunday to Saturday
  * For both customers, the months with highest rides are June, July, August,
  and September
  * Member rides are higher than casuals almost all day, except evening
  * Casual riders prefer electric bikes over classic bikes while members the
  opposite.

***

## **Share**

In this phase, we create compelling data visualizations that encompasses our
findings

### Data visualizations

We created different data visualizations. Next, we briefly describe each one.

  * Top 10 stations where members start a ride: contains the top stations that
  members frequently start a ride and the values for casual customers
  counterpart
  * Top 10 stations where casuals start a ride: presents the top stations where
  casual customers start a ride and the corresponding values for members
  * Summary of the dataset: it is a pivot table with five fields summarizing
  data of each type of member
  * Riders behavior during the week: there is a line for each type of member and
  the percent is related to the number of rides for each month
  * Frequency of each rideable: each circle represents the bike type, the color
  is whether casual or member customer, and the size is the percent of rides for
  each membership
  * Frequency of rides by month: each line is for a different membership, the
  data presented is for the last 12 months, and the percent is the portion of
  rides for each membership by month
  * Rides during the day: the colors represents the types of membership and the
  size of each pie chart is the amount of rides at each period

In [1]:
library("shinyLP")
iframe("100%", 700, "https://public.tableau.com/views/CyclisticBike-Share/
Top10stationswheremembersstartaride?:language=en-US&:display_count=n&:
origin=viz_share_link:showVizHome=no&:embed=true")

Shiny tags cannot be represented in plain text (need html)

### Conclusion

As a result of the data analysis process, we point out the key findings:

  * Customers are mostly concentrated in different stations
  * Casual riders takes longer rides on average, but members use bikes more often
  * Members use bikes regularly during the week, casuals ride on the weekend
  
***

## **Act**

This step is dedicated to gather insights and decide how best to implement them.

###  Call to action
  
  * Reach casual customers on the weekend, preferably Saturday afternoon.
  * Do marketing campaigns at casual riders top stations near parks
  * Create customer acquisition programs for June to September

***

#### **Let's ride a bike!**